In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta

In [3]:
file = 'D:/test\isc_radius_2024_06_15.parquet\part-00000-bf51a9ad-48d0-403c-8b84-56f38ace49db-c000.snappy.parquet'
df_raw = pd.read_parquet(file)
df_raw

,Name,IPAddress,CallerID,NASName,StatusType,Date,last_logon,last_logoff,SessID,SessTime,prev_uptime,alive_time,is_logon,is_logoff
0,Agdsl-091225-140,21.233.11.132,5c:1a:6f:bb:c1:cc,AGG-MP-01-02,ACTALIVE,Jun 15 2024 10:15:47,NaT,NaT,6248917840,971996,NaN,971996,0,0
1,Agdsl-091225-140,21.233.11.132,5c:1a:6f:bb:c1:cc,AGG-MP-01-02,ACTALIVE,Jun 15 2024 16:15:47,NaT,NaT,6248917840,993595,971996.0,21599,0,0
2,Agdsl-091225-140,21.233.11.132,5c:1a:6f:bb:c1:cc,AGG-MP-01-02,ACTALIVE,Jun 15 2024 22:15:47,NaT,NaT,6248917840,1015195,993595.0,21600,0,0
3,Agdsl-091225-140,21.233.11.132,5c:1a:6f:bb:c1:cc,AGG-MP-01-02,ACTALIVE,Jun 16 2024 04:15:47,NaT,NaT,6248917840,1036795,1015195.0,21600,0,0
4,Agdsl-100506-585,21.199.34.7,5c:1a:6f:f7:6f:ea,AGG-BRAS-CENTER-01-,ACTALIVE,Jun 15 2024 07:50:30,NaT,NaT,3237595,367200,NaN,367200,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297423,ybfdl-220511-0893,100.68.224.46,7c:9f:07:85:83:36,YBI-MP-01-02,ACTALIVE,Jun 16 2024 01:14:48,2024-06-15 00:14:39,NaT,35574105,64800,43200.0,21600,0,0
297424,ybfdl-221031-2018,100.68.232.125,7c:9f:07:85:75:86,YBI-MP-01-02,ACTALIVE,Jun 15 2024 07:34:17,NaT,NaT,29527849,885600,NaN,885600,0,0
297425,ybfdl-221031-2018,100.68.232.125,7c:9f:07:85:75:86,YBI-MP-01-02,ACTALIVE,Jun 15 2024 13:33:43,NaT,NaT,29527849,907200,885600.0,21600,0,0
297426,ybfdl-221031-2018,100.68.232.125,7c:9f:07:85:75:86,YBI-MP-01-02,ACTALIVE,Jun 15 2024 19:34:17,NaT,NaT,29527849,928800,907200.0,21600,0,0


In [4]:
df_session_info = df_raw[["Name", "SessID", "Date", "StatusType"]]
df_session_info

,Name,SessID,Date,StatusType
0,Agdsl-091225-140,6248917840,Jun 15 2024 10:15:47,ACTALIVE
1,Agdsl-091225-140,6248917840,Jun 15 2024 16:15:47,ACTALIVE
2,Agdsl-091225-140,6248917840,Jun 15 2024 22:15:47,ACTALIVE
3,Agdsl-091225-140,6248917840,Jun 16 2024 04:15:47,ACTALIVE
4,Agdsl-100506-585,3237595,Jun 15 2024 07:50:30,ACTALIVE
...,...,...,...,...
297423,ybfdl-220511-0893,35574105,Jun 16 2024 01:14:48,ACTALIVE
297424,ybfdl-221031-2018,29527849,Jun 15 2024 07:34:17,ACTALIVE
297425,ybfdl-221031-2018,29527849,Jun 15 2024 13:33:43,ACTALIVE
297426,ybfdl-221031-2018,29527849,Jun 15 2024 19:34:17,ACTALIVE


In [5]:
df_session_info['Name'].nunique()

62106

In [6]:
df_session_info['Date'] = pd.to_datetime(df_session_info['Date'])

In [7]:
def round_up_to_hour(dt):
    if dt.minute == 0 and dt.second == 0 and dt.microsecond == 0:
        return dt
    return dt + timedelta(hours=1, minutes=-dt.minute, seconds=-dt.second, microseconds=-dt.microsecond)

In [9]:
df_session_info['non_ovl'] = 0

def missing(df):
    for name, group in df.groupby(['Name']):
    
        group = group.sort_values('Date')
    
        unique_ids = group['SessID'].unique()
    
        for current_id in unique_ids:
            current_group = group[group['SessID'] == current_id]
            max_current_date = current_group['Date'].max()
            rounded_max_current_date = round_up_to_hour(max_current_date)
        
            for compare_id in unique_ids:
                if compare_id != current_id:
                    compare_group = group[group['SessID'] == compare_id]
                    min_compare_date = compare_group['Date'].min()
                    rounded_min_compare_date = round_up_to_hour(min_compare_date)
                
                    if max_current_date < min_compare_date and rounded_max_current_date == rounded_min_compare_date:
                        df.loc[(df['Name'] == name) & (df['SessID'] == current_id) & (df['Date'] == max_current_date), 'non_ovl'] +=1

    return(df)

df_session_info = missing(df_session_info)

print(df_session_info)

                     Name      SessID                Date StatusType  non_ovl
0        Agdsl-091225-140  6248917840 2024-06-15 10:15:47   ACTALIVE        0
1        Agdsl-091225-140  6248917840 2024-06-15 16:15:47   ACTALIVE        0
2        Agdsl-091225-140  6248917840 2024-06-15 22:15:47   ACTALIVE        0
3        Agdsl-091225-140  6248917840 2024-06-16 04:15:47   ACTALIVE        0
4        Agdsl-100506-585     3237595 2024-06-15 07:50:30   ACTALIVE        0
...                   ...         ...                 ...        ...      ...
297423  ybfdl-220511-0893    35574105 2024-06-16 01:14:48   ACTALIVE        0
297424  ybfdl-221031-2018    29527849 2024-06-15 07:34:17   ACTALIVE        0
297425  ybfdl-221031-2018    29527849 2024-06-15 13:33:43   ACTALIVE        0
297426  ybfdl-221031-2018    29527849 2024-06-15 19:34:17   ACTALIVE        0
297427  ybfdl-221031-2018    29527849 2024-06-16 01:33:52   ACTALIVE        0

[297428 rows x 5 columns]


In [11]:
def add_active_intervals(df, freq='H'):
    new_rows = []
    
    for (name, sessid), group in df_session_info.groupby(['Name', 'SessID']):
        group = group.sort_values(by='Date')
        
        for i in range(len(group) - 1):
            start_time = group.iloc[i]['Date']
            end_time = group.iloc[i + 1]['Date']
            
            if end_time > start_time:
                additional_times = pd.date_range(start=start_time + pd.Timedelta(hours=1),
                                                  end=end_time - pd.Timedelta(hours=1),
                                                  freq=freq)
                for time in additional_times:
                    new_rows.append({'Name': name, 'SessID': sessid, 'Date': time, 'StatusType': 'ACTALIVE', 'non_ovl': 0})
    
    additional_df = pd.DataFrame(new_rows)
    df_combined = pd.concat([df, additional_df]).sort_values(by=['Name','SessID','Date']).reset_index(drop=True)
    return df_combined

df_combined = add_active_intervals(df_session_info)

print(df_combined)

                      Name      SessID                Date StatusType  non_ovl
0         Agdsl-091225-140  6248917840 2024-06-15 10:15:47   ACTALIVE        0
1         Agdsl-091225-140  6248917840 2024-06-15 11:15:47   ACTALIVE        0
2         Agdsl-091225-140  6248917840 2024-06-15 12:15:47   ACTALIVE        0
3         Agdsl-091225-140  6248917840 2024-06-15 13:15:47   ACTALIVE        0
4         Agdsl-091225-140  6248917840 2024-06-15 14:15:47   ACTALIVE        0
...                    ...         ...                 ...        ...      ...
1128481  ybfdl-221031-2018    29527849 2024-06-15 20:34:17   ACTALIVE        0
1128482  ybfdl-221031-2018    29527849 2024-06-15 21:34:17   ACTALIVE        0
1128483  ybfdl-221031-2018    29527849 2024-06-15 22:34:17   ACTALIVE        0
1128484  ybfdl-221031-2018    29527849 2024-06-15 23:34:17   ACTALIVE        0
1128485  ybfdl-221031-2018    29527849 2024-06-16 01:33:52   ACTALIVE        0

[1128486 rows x 5 columns]


In [12]:
df_combined['date'] = df_combined['Date'].apply(round_up_to_hour)
df_combined

,Name,SessID,Date,StatusType,non_ovl,date
0,Agdsl-091225-140,6248917840,2024-06-15 10:15:47,ACTALIVE,0,2024-06-15 11:00:00
1,Agdsl-091225-140,6248917840,2024-06-15 11:15:47,ACTALIVE,0,2024-06-15 12:00:00
2,Agdsl-091225-140,6248917840,2024-06-15 12:15:47,ACTALIVE,0,2024-06-15 13:00:00
3,Agdsl-091225-140,6248917840,2024-06-15 13:15:47,ACTALIVE,0,2024-06-15 14:00:00
4,Agdsl-091225-140,6248917840,2024-06-15 14:15:47,ACTALIVE,0,2024-06-15 15:00:00
...,...,...,...,...,...,...
1128481,ybfdl-221031-2018,29527849,2024-06-15 20:34:17,ACTALIVE,0,2024-06-15 21:00:00
1128482,ybfdl-221031-2018,29527849,2024-06-15 21:34:17,ACTALIVE,0,2024-06-15 22:00:00
1128483,ybfdl-221031-2018,29527849,2024-06-15 22:34:17,ACTALIVE,0,2024-06-15 23:00:00
1128484,ybfdl-221031-2018,29527849,2024-06-15 23:34:17,ACTALIVE,0,2024-06-16 00:00:00


In [14]:
df_non_overlap = df_combined[['Name','date', 'non_ovl']].groupby(['Name', 'date'])['non_ovl'].agg(sum).reset_index()
df_non_overlap

,Name,date,non_ovl
0,Agdsl-091225-140,2024-06-15 11:00:00,0
1,Agdsl-091225-140,2024-06-15 12:00:00,0
2,Agdsl-091225-140,2024-06-15 13:00:00,0
3,Agdsl-091225-140,2024-06-15 14:00:00,0
4,Agdsl-091225-140,2024-06-15 15:00:00,0
...,...,...,...
1079845,ybfdl-221031-2018,2024-06-15 21:00:00,0
1079846,ybfdl-221031-2018,2024-06-15 22:00:00,0
1079847,ybfdl-221031-2018,2024-06-15 23:00:00,0
1079848,ybfdl-221031-2018,2024-06-16 00:00:00,0


In [22]:
df_actalive = df_combined[df_combined['StatusType'] == 'ACTALIVE']
total_actalive = df_actalive.groupby(['Name', 'date'])['SessID'].nunique().reset_index()
total_actalive = total_actalive.rename(columns={'SessID': 'total'})
total_actalive

,Name,date,total
0,Agdsl-091225-140,2024-06-15 11:00:00,1
1,Agdsl-091225-140,2024-06-15 12:00:00,1
2,Agdsl-091225-140,2024-06-15 13:00:00,1
3,Agdsl-091225-140,2024-06-15 14:00:00,1
4,Agdsl-091225-140,2024-06-15 15:00:00,1
...,...,...,...
1075280,ybfdl-221031-2018,2024-06-15 21:00:00,1
1075281,ybfdl-221031-2018,2024-06-15 22:00:00,1
1075282,ybfdl-221031-2018,2024-06-15 23:00:00,1
1075283,ybfdl-221031-2018,2024-06-16 00:00:00,1


In [24]:
df_cheat = pd.merge(total_actalive, df_non_overlap, on=['Name', 'date'], how='left')
df_cheat['overlap'] = df_cheat['total'] - df_cheat['non_ovl']
df_cheat

,Name,date,total,non_ovl,overlap
0,Agdsl-091225-140,2024-06-15 11:00:00,1,0,1
1,Agdsl-091225-140,2024-06-15 12:00:00,1,0,1
2,Agdsl-091225-140,2024-06-15 13:00:00,1,0,1
3,Agdsl-091225-140,2024-06-15 14:00:00,1,0,1
4,Agdsl-091225-140,2024-06-15 15:00:00,1,0,1
...,...,...,...,...,...
1075280,ybfdl-221031-2018,2024-06-15 21:00:00,1,0,1
1075281,ybfdl-221031-2018,2024-06-15 22:00:00,1,0,1
1075282,ybfdl-221031-2018,2024-06-15 23:00:00,1,0,1
1075283,ybfdl-221031-2018,2024-06-16 00:00:00,1,0,1


In [25]:
df_cheat['date_only'] = df_cheat['date'].dt.date
df_cheat['hour'] = df_cheat['date'].dt.hour
print(df_cheat)

                      Name                date  total  non_ovl  overlap  \
0         Agdsl-091225-140 2024-06-15 11:00:00      1        0        1   
1         Agdsl-091225-140 2024-06-15 12:00:00      1        0        1   
2         Agdsl-091225-140 2024-06-15 13:00:00      1        0        1   
3         Agdsl-091225-140 2024-06-15 14:00:00      1        0        1   
4         Agdsl-091225-140 2024-06-15 15:00:00      1        0        1   
...                    ...                 ...    ...      ...      ...   
1075280  ybfdl-221031-2018 2024-06-15 21:00:00      1        0        1   
1075281  ybfdl-221031-2018 2024-06-15 22:00:00      1        0        1   
1075282  ybfdl-221031-2018 2024-06-15 23:00:00      1        0        1   
1075283  ybfdl-221031-2018 2024-06-16 00:00:00      1        0        1   
1075284  ybfdl-221031-2018 2024-06-16 02:00:00      1        0        1   

          date_only  hour  
0        2024-06-15    11  
1        2024-06-15    12  
2        2024-0

In [29]:
names = df_cheat['Name'].unique()
dates = df_cheat['date_only'].unique()
hours = list(range(24))  

all_combinations = pd.MultiIndex.from_product([names, dates, hours], names=['Name', 'date_only', 'hour']).to_frame(index=False)

print(all_combinations)

                      Name   date_only  hour
0         Agdsl-091225-140  2024-06-15     0
1         Agdsl-091225-140  2024-06-15     1
2         Agdsl-091225-140  2024-06-15     2
3         Agdsl-091225-140  2024-06-15     3
4         Agdsl-091225-140  2024-06-15     4
...                    ...         ...   ...
2978443  ybfdl-221031-2018  2024-06-16    19
2978444  ybfdl-221031-2018  2024-06-16    20
2978445  ybfdl-221031-2018  2024-06-16    21
2978446  ybfdl-221031-2018  2024-06-16    22
2978447  ybfdl-221031-2018  2024-06-16    23

[2978448 rows x 3 columns]


In [30]:
df_full = pd.merge(all_combinations, df_cheat, on=['Name', 'date_only', 'hour'], how='left')
df_full = df_full.drop('date', axis=1)
df_full['total'] = df_full['total'].fillna(0).astype(int)
df_full['non_ovl'] = df_full['non_ovl'].fillna(0).astype(int)
df_full['overlap'] = df_full['overlap'].fillna(0).astype(int)
df_full

                      Name   date_only  hour  total  non_ovl  overlap
0         Agdsl-091225-140  2024-06-15     0      0        0        0
1         Agdsl-091225-140  2024-06-15     1      0        0        0
2         Agdsl-091225-140  2024-06-15     2      0        0        0
3         Agdsl-091225-140  2024-06-15     3      0        0        0
4         Agdsl-091225-140  2024-06-15     4      0        0        0
...                    ...         ...   ...    ...      ...      ...
2978443  ybfdl-221031-2018  2024-06-16    19      0        0        0
2978444  ybfdl-221031-2018  2024-06-16    20      0        0        0
2978445  ybfdl-221031-2018  2024-06-16    21      0        0        0
2978446  ybfdl-221031-2018  2024-06-16    22      0        0        0
2978447  ybfdl-221031-2018  2024-06-16    23      0        0        0

[2978448 rows x 6 columns]


In [37]:
df_full[df_full['overlap'] == 3 ].sort_values(by = ['Name', 'date_only', 'hour'])

,Name,date_only,hour,total,non_ovl,overlap
20793,Agfdl-220820-8671,2024-06-15,9,3,0,3
45813,Bdfdl-170728-652,2024-06-15,21,4,1,3
383877,Dafdl-230927-2882,2024-06-15,21,3,0,3
893874,Hnfdl-200408-892,2024-06-15,18,3,0,3
941309,Hnfdl-210214-730,2024-06-16,5,3,0,3
...,...,...,...,...,...,...
2767082,hnfdl-210818-725,2024-06-16,2,3,0,3
2767083,hnfdl-210818-725,2024-06-16,3,3,0,3
2767085,hnfdl-210818-725,2024-06-16,5,3,0,3
2773998,hnfdl-220224-0345,2024-06-16,6,3,0,3
